In [54]:
!pip install torch
!pip install -U openmim
!mim install mmengine
!mim install 'mmdet>=3.0.0'
!mim install "mmdet3d>=1.1.0"



Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.2.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.2.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.2.0/index.html


In [55]:
from mmdet3d.apis import LidarDet3DInferencer

In [56]:
# initialize inferencer
inferencer = LidarDet3DInferencer('pointpillars_kitti-3class')

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection3d/v1.0.0_models/pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth


/usr/local/lib/python3.10/dist-packages/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [57]:
# inference
inputs = dict(points='/content/000008.bin')
predictions = inferencer(inputs)
print(predictions)

Output()

{'predictions': [{'labels_3d': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2], 'scores_3d': [0.9519277215003967, 0.9417073726654053, 0.9156165719032288, 0.898225724697113, 0.8714114427566528, 0.7962632775306702, 0.7739599347114563, 0.6636926531791687, 0.6451439261436462, 0.5039315819740295], 'bboxes_3d': [[14.76012134552002, -1.1046630144119263, -1.5373586416244507, 3.7437565326690674, 1.5428502559661865, 1.4906117916107178, -0.31211209297180176], [8.108292579650879, 1.2249923944473267, -1.5887396335601807, 3.6825459003448486, 1.5752863883972168, 1.5501244068145752, 2.8276779651641846], [6.441560745239258, -3.876326560974121, -1.7240561246871948, 3.1512696743011475, 1.507940411567688, 1.4468772411346436, -0.32760751247406006], [4.030516624450684, 2.6715595722198486, -1.6319200992584229, 3.3710806369781494, 1.5763121843338013, 1.6236366033554077, -0.27743709087371826], [33.607032775878906, -7.086616039276123, -1.3757083415985107, 4.336166858673096, 1.7496241331100464, 1.7403538227081299, 2.867822647094

In [58]:
import numpy as np

# Parse predictions

# Parse ground truth annotations
ground_truth_annotations = """Car 0.88 3 -0.69 0.00 192.37 402.31 374.00 1.60 1.57 3.23 -2.70 1.74 3.68 -1.29
Car 0.00 1 2.04 334.85 178.94 624.50 372.04 1.57 1.50 3.68 -1.17 1.65 7.86 1.90
Car 0.34 3 -1.84 937.29 197.39 1241.00 374.00 1.39 1.44 3.08 3.81 1.64 6.15 -1.31
Car 0.00 1 -1.33 597.59 176.18 720.90 261.14 1.47 1.60 3.66 1.07 1.55 14.44 -1.25
Car 0.00 0 1.74 741.18 168.83 792.25 208.43 1.70 1.63 4.08 7.24 1.55 33.20 1.95
Car 0.00 0 -1.65 884.52 178.31 956.41 240.18 1.59 1.59 2.47 8.48 1.75 19.96 -1.25
DontCare -1 -1 -10 800.38 163.67 825.45 184.07 -1 -1 -1 -1000 -1000 -1000 -10
DontCare -1 -1 -10 859.58 172.34 886.26 194.51 -1 -1 -1 -1000 -1000 -1000 -10
DontCare -1 -1 -10 801.81 163.96 825.20 183.59 -1 -1 -1 -1000 -1000 -1000 -10
DontCare -1 -1 -10 826.87 162.28 845.84 178.86 -1 -1 -1 -1000 -1000 -1000 -10"""

# Convert ground truth annotations to list of lists
ground_truth_boxes = [line.split()[1:] for line in ground_truth_annotations.split('\n') if line.strip()]
# Function to calculate IoU
import numpy as np
from sklearn.metrics import accuracy_score
import torch

def cosine_similarity(array1, array2):
    """
    Calculate cosine similarity between two numpy arrays.

    Parameters:
        array1 (numpy.ndarray): First input array.
        array2 (numpy.ndarray): Second input array.

    Returns:
        float: Cosine similarity between the two input arrays.
    """
    dot_product = np.dot(array1, array2)
    norm1 = np.linalg.norm(array1)
    norm2 = np.linalg.norm(array2)

    if norm1 == 0 or norm2 == 0:
        return 0.0

    return dot_product / (norm1 * norm2)


def calculate_accuracy(predictions, ground_truth_boxes, threshold=0.00000000001):
    # List to store accuracy values
    accuracies = []

    # Iterate over predictions
    for prediction in predictions['predictions']:
        for pred_box in prediction['bboxes_3d']:
            max_accuracy = 0
            for gt_box in ground_truth_boxes:
                # Calculate the absolute difference between prediction and ground truth
                # Extract bounding box properties
                l1, w1, h1 = float(pred_box[0]), float(pred_box[1]), float(pred_box[2])  # length, width, height of box1
                l2, w2, h2 = float(gt_box[12]), float(gt_box[11]), float(gt_box[10])  # length, width, height of box2

    # Calculate box coordinates
                x1, y1, z1 = float(pred_box[3]), float(pred_box[4]), float(pred_box[5])  # x, y, z of box1
                x2, y2, z2 = float(gt_box[9]), float(gt_box[8]), float(gt_box[7])  # x, y, z of box2
                b1 = np.array([l1,w1,h1,x1,y1,z1])
                b2 = np.array([l2,w2,h2,x2,y2,z2])
                cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

                accuracy =  cosine_similarity(b1,b2)
                # Update max accuracy
                if accuracy > max_accuracy:
                    max_accuracy = accuracy
                    #print(l1,w1,h1,)

            accuracies.append(max_accuracy)

    # Calculate overall accuracy
    overall_accuracy = np.mean(accuracies)
    return overall_accuracy

# Calculate accuracy
accuracy = calculate_accuracy(predictions, ground_truth_boxes)
print("Accuracy:", accuracy)


Accuracy: 0.9211952381694045
